<a href="https://colab.research.google.com/github/cpython-projects/da_1709/blob/main/lesson_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Шкалы

## Номинальная шкала (Nominal Scale)

**Определение:**  
Номинальная шкала — это **категориальные данные без упорядочивания**.  
Она используется для классификации объектов по группам, не имеющим количественного или логического порядка.

Примеры:
- Цвета: `red`, `blue`, `green`
- Тип продукта: `smartphone`, `laptop`
- Страна: `Ukraine`, `Germany`, `Japan`
- Пол: `male`, `female`

**Ключевые особенности:**
- Значения не имеют математического смысла (нельзя сложить или усреднить цвета)
- Можно вычислять только частоту (моду) — самое частое значение
- Основные способы визуализации: **круговые диаграммы** и **столбчатые графики**

In [2]:
import pandas as pd

# Пример: данные о клиентах и их предпочтениях по цвету и типу продукта
data = {
    'Color': ['red', 'blue', 'green', 'blue', 'blue', 'red', 'green', 'red', 'red', 'blue'],
    'Product': ['smartphone', 'laptop', 'smartphone', 'tablet', 'laptop', 'tablet', 'laptop', 'smartphone', 'tablet', 'laptop'],
    'Country': ['Ukraine', 'Germany', 'Japan', 'Ukraine', 'Japan', 'Germany', 'Ukraine', 'Japan', 'Ukraine', 'Germany'],
    'Gender': ['male', 'female', 'female', 'male', 'male', 'female', 'male', 'female', 'female', 'male']
}

df = pd.DataFrame(data)
df.head()

,Color,Product,Country,Gender
0,red,smartphone,Ukraine,male
1,blue,laptop,Germany,female
2,green,smartphone,Japan,female
3,blue,tablet,Ukraine,male
4,blue,laptop,Japan,male


## Порядковая шкала (Ordinal Scale)

**Определение:**  
Порядковая шкала — это **категориальные данные с упорядочением**,  
но **интервалы между значениями не равны и не поддаются точному измерению**.

Примеры:
- Уровень образования: `primary < secondary < tertiary`
- Оценка продукта: `1★ < 2★ < 5★`
- Стадия болезни: `mild < moderate < severe`
- Уровень дохода: `low < medium < high`

**Ключевые особенности:**
- Порядок **имеет значение**, но разница между категориями **не количественна**.  
- Подходящие статистики: **медиана**, **ранг**.  
- Визуализация: **упорядоченные bar-графики**, **Likert шкалы**.


In [5]:
# Пример данных: уровень удовлетворённости клиентов
data = {
    'Satisfaction': ['very low', 'low', 'medium', 'high', 'very high', 'medium', 'low', 'medium', 'high', 'very high']
}

df = pd.DataFrame(data)
df

,Satisfaction
0,very low
1,low
2,medium
3,high
4,very high
5,medium
6,low
7,medium
8,high
9,very high


In [6]:
# Указываем порядок категорий вручную (от низкого к высокому)
order = ['very low', 'low', 'medium', 'high', 'very high']
df['Satisfaction'] = pd.Categorical(df['Satisfaction'], categories=order, ordered=True)
df

,Satisfaction
0,very low
1,low
2,medium
3,high
4,very high
5,medium
6,low
7,medium
8,high
9,very high


### Если порядок **НЕ задан** (`ordered=False`, значение по умолчанию)

Pandas просто хранит набор меток (labels), **без знания, какая "выше" или "ниже"**.
То есть `“low”` и `“high”` будут рассматриваться просто как разные категории — **без логического порядка**.

In [7]:
df = pd.DataFrame({'Satisfaction': ['low', 'medium', 'high']})
df['Satisfaction'] = df['Satisfaction'].astype('category')
print(df['Satisfaction'].cat.ordered)

False


➡️ При этом:

* сортировка (`sort_values`) идёт **по алфавиту**, а не по смысловому порядку;
* нельзя сравнивать (`<`, `>`) категории — вызовет ошибку;
* медиану или ранги корректно вычислить нельзя.

### Если порядок **ЗАДАН** (`ordered=True`)

Теперь pandas понимает, что `very low < low < medium < high < very high`.

In [8]:
order = ['very low', 'low', 'medium', 'high', 'very high']
df['Satisfaction'] = pd.Categorical(df['Satisfaction'], categories=order, ordered=True)
print(df['Satisfaction'].cat.ordered)

True


Теперь возможны:

* **сравнения:**

  ```python
  df['Satisfaction'] > 'medium'
  ```

  вернёт логические значения;
* **корректная сортировка:**

  ```python
  df.sort_values('Satisfaction')
  ```

  упорядочит данные не по алфавиту, а по смыслу;
* **расчёт рангов и медианы по смысловому порядку.**

In [9]:
df['Satisfaction'] > 'medium'

,Satisfaction
0,False
1,False
2,True


In [10]:
df.sort_values('Satisfaction')

,Satisfaction
0,low
1,medium
2,high


### Краткий вывод

| Сценарий                              | ordered=False | ordered=True |
| ------------------------------------- | ------------- | ------------ |
| Можно сортировать по смыслу           | ❌             | ✅            |
| Можно сравнивать категории (`>`, `<`) | ❌             | ✅            |
| Медиана/ранг имеет смысл              | ❌             | ✅            |
| Подходит для номинальных данных       | ✅             | ❌            |
| Подходит для порядковых данных        | ❌             | ✅            |

---

### Рекомендация

* Для **Nominal Scale** (`цвет`, `страна`, `тип товара`) → `ordered=False`
* Для **Ordinal Scale** (`low`, `medium`, `high`) → `ordered=True`

## Количественная шкала (Quantitative Scale)

**Определение:**  
Количественная шкала используется для **числовых данных, имеющих математический смысл**.  
Она позволяет выполнять арифметические операции: сложение, вычитание, умножение и деление.  

**Подтипы:**
- **Дискретные (целые значения)** — количество товаров, число детей в семье.  
- **Непрерывные (дробные значения)** — вес, рост, температура, время доставки.

**Примеры:**
- Возраст: `25`, `30.5`
- Зарплата: `$50,000`
- Время доставки: `2.5 часа`
- Просмотры сайта: `1,000,000`

**Ключевые особенности:**
- Применимы все математические операции (`+`, `−`, `×`, `÷`)
- Основные статистики: **среднее, стандартное отклонение, дисперсия**
- Типичные визуализации: **гистограммы**, **boxplots**, **scatter plots**


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Пример количественных данных: возраст, зарплата, время доставки
data = {
    'Age': [23, 29, 35, 42, 45, 50, 56, 61, 65, 70],
    'Salary': [3000, 3500, 4000, 4500, 4800, 5200, 6000, 7000, 7200, 7500],
    'Delivery_time': [2.1, 2.5, 2.3, 2.0, 2.6, 2.8, 3.0, 2.9, 3.2, 2.7]
}

df = pd.DataFrame(data)
df


,Age,Salary,Delivery_time
0,23,3000,2.1
1,29,3500,2.5
2,35,4000,2.3
3,42,4500,2.0
4,45,4800,2.6
5,50,5200,2.8
6,56,6000,3.0
7,61,7000,2.9
8,65,7200,3.2
9,70,7500,2.7


## Сравнение типов шкал данных (Comparison of Scales)

| Критерий | Nominal | Ordinal | Quantitative |
|-----------|----------|----------|--------------|
| **Наличие порядка (Order)** | ✘ Нет | ✔ Есть | ✔ Есть |
| **Равные интервалы (Equal Intervals)** | ✘ Нет | ✘ Нет | ✔ Да |
| **Математические операции (Math Operations)** | ✘ Не применимы | ✘ Ограничены (только ранги) | ✔ Все арифметические операции |
| **Основные статистики (Statistics)** | Мода, частота | Медиана, мода, ранги | Среднее, медиана, мода, дисперсия, стандартное отклонение |
| **Примеры (Examples)** | Пол, цвета, страны | Рейтинги, образование, стадии болезни | Вес, рост, температура, возраст |

---

### Важные замечания
- ⚠️ **Типичная ошибка:** вычислять среднее значение для порядковых данных (например, "средняя оценка = 3.8") —  
  математически некорректно, потому что интервалы между категориями неравны.  
- ✅ **Правило:** выбор статистических методов и визуализаций **зависит от типа шкалы**.  
  Перед анализом всегда проверяйте, соответствует ли ваш метод типу данных.

# Сводная статистика

**Понятие (Concept):**  
Сводная статистика — это часть описательной статистики, которая предоставляет **краткое числовое резюме данных**.  
Она помогает быстро понять, как распределены значения, где центр и каков разброс.  
Сводные показатели (например, среднее, медиана, стандартное отклонение) описывают основные характеристики выборки.

**Почему это важно (Why is it important?):**  
- Помогает понять общую структуру данных.  
- Позволяет выявить закономерности, тенденции и потенциальные проблемы (например, пропуски или выбросы).  
- Создаёт основу для дальнейшего анализа, визуализации или построения моделей машинного обучения.

**Примеры сводных статистик:**
- Среднее значение (`mean`)  
- Медиана (`median`)  
- Мода (`mode`)  
- Минимум и максимум (`min`, `max`)  
- Дисперсия (`variance`)  
- Стандартное отклонение (`std`)